# Necessary Installations and Imports

In [11]:
!pip install EasyGA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.2 MB/s eta 0:00:00
  Created wheel for matplotlib: filename=matplotlib-3.3.4-cp310-cp310-linux_x86_64.whl size=11684775 sha256=6d86c78bca09fbe087ec1bd4891cad82d447c0ebf088323835d084f52b78d539
  Stored in directory: /root/.cache/pip/wheels/38/c6/49/eaba6d234887d98d9c85185e2a90bd7bb77934e85eefaf317e
Successfully built matplotlib
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart

In [1]:
!pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=856763dff4a2a6fb25311c451f37ed8b56d0bf6a7e441456413e8f8014521ec6
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [2]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np

# Automated Pipeline to calculate Fuzy Tip using Python class

In [13]:
class FuzzyTip:

  def __init__(self, inputs):

    # Instantiating user inputs
    self.inp_temperature = inputs['temperature']
    self.inp_flavour = inputs['flavor']
    self.inp_portion_size = inputs['portion_size']
    self.inp_attentiveness = inputs['attentiveness']
    self.inp_friendliness = inputs['friendliness']
    self.inp_speed = inputs['speed']

    #declaring inputs and outputs of the Fuzzy Logic System i.e. Fuzzy Variables
    # Level 1 inputs
    self.temperature = ctrl.Antecedent(np.linspace(0, 10, 11), 'temperature')
    self.flavour = ctrl.Antecedent(np.linspace(0, 10, 11), 'flavour')
    self.portion_size = ctrl.Antecedent(np.linspace(0, 10, 11), 'portion_size')
    self.attentiveness = ctrl.Antecedent(np.linspace(0, 10, 11), 'attentiveness')
    self.friendliness = ctrl.Antecedent(np.linspace(0, 10, 11), 'friendliness')
    self.speed = ctrl.Antecedent(np.linspace(0, 10, 11), 'speed')
    # Level 1 outputs
    self.food_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'food_quality')
    self.service_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'service_quality')
    # Level 2 inputs
    self.food_quality_for_tip = ctrl.Antecedent(np.linspace(0, 10, 11), 'food_quality_for_tip')
    self.service_quality_for_tip = ctrl.Antecedent(np.linspace(0, 10, 11), 'service_quality_for_tip')
    # Level 2 output
    self.tip = ctrl.Consequent(np.linspace(0, 25, 26), 'tip')

    #fuzzy var
    self.fuzzy_var = {
        0: "temperature",
        1: "flavour",
        2: "portion_size",
        3: "attentiveness",
        4: "friendliness",
        5: "speed",
        6: "food_quality",
        7: "service_quality",
        8: "food_quality_for_tip",
        9: "service_quality_for_tip",
        10: "tip"
    }


  def membership_function(self, chromosome):
    # List comprehension
    chromosome_list = [[int(gene.value) for gene in chromosome[i:i+3]] for i in range(0, len(chromosome), 3)]
    chromosome_list = [chromosome_list[i:i+3] for i in range(0, len(chromosome_list), 3)]
    for key, val in self.fuzzy_var.items():
      abc = chromosome_list[key]
      print(abc)
      poor = fuzz.trimf(getattr(self, val).universe, sorted(abc[0]))
      average = fuzz.trimf(getattr(self, val).universe, sorted(abc[1]))
      good = fuzz.trimf(getattr(self, val).universe, sorted(abc[2]))
      my_dict={}
      keys = ['poor', 'average', 'good']
      values = [poor, average, good]
      for key, value in zip(keys, values):
        my_dict[key] = value
      setattr(self, val, my_dict)


  # def membership_function(self):
  #   self.temperature['poor'] = fuzz.trimf(self.temperature.universe, [0, 0, 5])
  #   self.temperature['average'] = fuzz.trimf(self.temperature.universe, [0, 5, 10])
  #   self.temperature['good'] = fuzz.trimf(self.temperature.universe, [5, 5, 10])

  def fuzzy_food_quality_rules(self):
    food_quality_rules = []
    encoding = {'poor': 0, 'average': 1, 'good': 2}
    metrics = ['poor', 'average', 'good']
    for t in metrics:
        for f in metrics:
            for p in metrics:
                # Define the rule for the current combination
                iter_val = []
                iter_val.append(encoding[t])
                iter_val.append(encoding[f])
                iter_val.append(encoding[p])
                avg_floor = (iter_val[0] + iter_val[1] + iter_val[2]) // 3
                if(avg_floor==0):
                  rule = ctrl.Rule(self.temperature[t] & self.flavour[f] & self.portion_size[p], self.food_quality['poor'])
                elif(avg_floor==1):
                  rule = ctrl.Rule(self.temperature[t] & self.flavour[f] & self.portion_size[p], self.food_quality['average'])
                else:
                  rule = ctrl.Rule(self.temperature[t] & self.flavour[f] & self.portion_size[p], self.food_quality['good'])
                # Add the rule to the list of rules
                food_quality_rules.append(rule)
    return food_quality_rules

  def fuzzy_service_quality_rules(self):
    service_quality_rules = []
    encoding = {'poor': 0, 'average': 1, 'good': 2}
    metrics = ['poor', 'average', 'good']
    for a in metrics:
        for f in metrics:
            for s in metrics:
                # Define the rule for the current combination
                iter_val = []
                iter_val.append(encoding[a])
                iter_val.append(encoding[f])
                iter_val.append(encoding[s])
                avg_floor = (iter_val[0] + iter_val[1] + iter_val[2]) // 3
                if avg_floor == 0:
                  rule = ctrl.Rule(self.attentiveness[a] & self.friendliness[f] & self.speed[s], self.service_quality['poor'])
                elif avg_floor == 1:
                  rule = ctrl.Rule(self.attentiveness[a] & self.friendliness[f] & self.speed[s], self.service_quality['average'])
                else:
                  rule = ctrl.Rule(self.attentiveness[a] & self.friendliness[f] & self.speed[s], self.service_quality['good'])
                # Add the rule to the list of rules
                service_quality_rules.append(rule)
    return service_quality_rules

  def level_one_fuzzy_tree(self):
    #Defining Control System and Simulation for layer one
    food_ctrl = ctrl.ControlSystem(self.fuzzy_food_quality_rules())
    food_sim = ctrl.ControlSystemSimulation(food_ctrl)
    service_ctrl = ctrl.ControlSystem(self.fuzzy_service_quality_rules())
    service_sim = ctrl.ControlSystemSimulation(service_ctrl)
    # Feeding inputs to the first level of fuzzy tree: food quality and service quality control systems
    food_sim.input['temperature'] = self.inp_temperature
    food_sim.input['flavour'] = self.inp_flavour
    food_sim.input['portion_size'] =self.inp_portion_size
    food_sim.compute()
    service_sim.input['attentiveness'] = self.inp_attentiveness
    service_sim.input['friendliness'] = self.inp_friendliness
    service_sim.input['speed'] = self.inp_speed
    service_sim.compute()
    # Retus Control Simulation objects
    return food_sim, service_sim

  def fuzzy_tip_rules(self):
    tip_rules = []
    metrics = ['poor', 'average', 'good']
    for f in metrics:
      for s in metrics:
        if(f=='good' and s =='good'):
          rule = ctrl.Rule(self.food_quality_for_tip[f] & self.service_quality_for_tip[s], self.tip['good'])
        elif((f=='average' or s =='average') and (f!='poor' and s!='poor')):
          rule = ctrl.Rule(self.food_quality_for_tip[f] & self.service_quality_for_tip[s], self.tip['average'])
        else:
          rule = ctrl.Rule(self.food_quality_for_tip[f] & self.service_quality_for_tip[s], self.tip['poor'])
        tip_rules.append(rule)
    return tip_rules

  def level_two_fuzzy_tree(self):
    # Getting Control System Simulation objects from layer one
    food_sim, service_sim = self.level_one_fuzzy_tree()
    # Defining Control System and Simulation for layer two
    tip_ctrl = ctrl.ControlSystem(self.fuzzy_tip_rules())
    tip_sim = ctrl.ControlSystemSimulation(tip_ctrl)
    # Feeding inputs to the second layer of the fuzzy tree: tip control system
    food_quality_output = food_sim.output['food_quality']
    service_quality_output = service_sim.output['service_quality']
    tip_sim.input['food_quality_for_tip'] = food_quality_output
    tip_sim.input['service_quality_for_tip'] = service_quality_output
    tip_sim.compute()
    return tip_sim

  def fuzzy_tip(self):
    tip_sim = self.level_two_fuzzy_tree()
    calculated_tip = tip_sim.output['tip']
    return calculated_tip

# Python Program for Calculating the Tip

In [ ]:
class InputOutOfRangeError(Exception):
    def __init__(self, message="Input is out of bounds. It should be in a number in the range 1 to 10."):
        super().__init__(message)


criteria = {0: "Temperature of food", 1: "Flavour of food",
            2: "Portion Size of food", 3: "Attentiveness during service",
            4: "Friendliness during service", 5:"Speed of Service"}
while True:
    # Gather user input for six measures
    print("Please rate your experience in terms of the following Food and Service Quality Measures between 1 to 10 (1=low to 10=high):")
    measures = [0]*6
    i = 0
    while True:
      try:
        measures[i] = float(input(f"{criteria[i]}: "))
        if i==5:
          break
        elif not (1 <= measures[i] <= 10):
          raise InputOutOfRangeError
        else:
          i = i + 1
      except InputOutOfRangeError as e:
        print(e)
        continue
      except ValueError:
        print("Invalid input. Please enter numeric values.")
        continue

    f = FuzzyTip(measures[0], measures[1], measures[2], measures[3], measures[4], measures[5])

    # Calculate the recommended tip
    tip_percentage = round(f.fuzzy_tip(),2)
    print(f"Recommended tip as a percentage is: {tip_percentage}%")

    # Ask the user if they want to enter another set of measures
    another_set = input("Do you want to enter another set of measures? (yes/no): ").strip().lower()
    if another_set != 'yes':
        break


Please rate your experience in terms of the following Food and Service Quality Measures between 1 to 10 (1=low to 10=high):


KeyboardInterrupt: ignored

In [ ]:
len(f.fuzzy_food_quality_rules())

In [ ]:
type(f.temperature['poor'](3))

# Genetic Algorithm part of the code


Train Data Ingestion and Cleaning

In [4]:
import pandas as pd

In [50]:
df = pd.read_csv('/content/tipper_train.csv', header=0)
df.iloc[:, :-1] = df.iloc[:, :-1].clip(0.0, 1.0)
train_data = df
train_data.columns = ['temperature', 'flavor', 'portion_size', 'attentiveness',
                      'friendliness', 'speed', 'tip']

In [52]:
input = (train_data.iloc[0]).to_dict()
print(type(input))
f = FuzzyTip(input)
f.temperature["poor"]

<class 'dict'>


ValueError: ignored

Defining how to generate chromosomes

In [ ]:
# class GeneticAlgo:
#   def generate_chromosome():
#     chromosome = [ga.gene_impl() for _ in range(ga.chromosome_length)]
#     return chromosome

In [7]:
# Defining how to generate chromosomes
# def generate_chromosome(ga):
#   chromosome = [ga.gene_impl() for i in range(ga.chromosome_length)]
#   return chromosome

# Defining how to generate chromosomes
def generate_chromosome():
  chromosome = random.randint(0, 10)
  return (chromosome)

In [ ]:
generate_chromosome()

9

In [8]:
def execute_fuzzy_inference(chromosome, inputs):
  f = FuzzyTip(inputs)
  print('setup fuzzy system')
  f.membership_function(chromosome)
  predicted_tip = f.fuzzy_tip()
  del f
  return predicted_tip

Defining the Ftness function

In [53]:
# Defining the fitness function
def fitness(chromosome):
  total_error = 0
  for index, row in train_data.iterrows():
    inputs = {
    'temperature': row['temperature'],
    'flavor': row['food flavor'],
    'portion_size': row['portion_size'],
    'attentiveness': row['attentiveness'],
    'friendliness': row['friendliness'],
    'speed': row['speed']
    }
    actual_tip = row['tip']
    predicted_tip = execute_fuzzy_inference(chromosome, inputs)
    error = abs(actual_tip - predicted_tip)
    total_error += error
  return total_error

In [14]:
import EasyGA
import random

# Create the Genetic algorithm
ga = EasyGA.GA()
# Chromosome parameters
ga.chromosome_length = 99
ga.population_size = 50

# Minimizing the total_error returned by our fitness() function
ga.target_fitness_type = 'min'

#No. of iterations to run
ga.generation_goal = 10

# Gene implementation: generating chromosomes
# ga.gene_impl = lambda: generate_chromosome(ga)
ga.gene_impl = lambda: generate_chromosome()
# chromosome = generate_chromosome(ga)

# fitness function
# ga.fitness_function_impl = fitness(ga.population.chromosome_list)
ga.fitness_function_impl = fitness

# Evolve the whole genetic algorithm until termination has been reached
ga.evolve()

# Print out the current generation and the population
ga.print_generation()
ga.print_population()

setup fuzzy system
[[3, 1, 8], [8, 9, 2], [2, 9, 10]]
[[1, 3, 9], [1, 7, 4], [0, 5, 0]]
[[4, 0, 9], [4, 3, 1], [8, 6, 0]]
[[1, 0, 5], [5, 6, 7], [6, 10, 4]]
[[10, 7, 10], [8, 5, 5], [2, 8, 6]]
[[0, 5, 3], [8, 5, 0], [5, 8, 9]]
[[0, 8, 7], [7, 7, 3], [8, 2, 1]]
[[0, 5, 5], [10, 0, 5], [4, 4, 8]]
[[7, 2, 4], [9, 6, 3], [5, 9, 5]]
[[8, 7, 1], [0, 4, 9], [5, 2, 2]]
[[8, 1, 5], [3, 4, 5], [8, 8, 2]]


TypeError: ignored